In [1]:
from common_imports import *
%matplotlib inline

Couldn't import dot_parser, loading of dot files will not be possible.


# PacMan SmallGrid Generator

In [31]:
def load_task(path):
    _layout = layout.getLayout(path)
    ghostType = DirectionalGhost
    agents = [ghostType( i+1 ) for i in range(2)]
    display = textDisplay.NullGraphics()
    task = PacmanTask(_layout, agents, display, state_repr='stack')
    return task
task0 = load_task('data/layouts/smallGrid.lay')
task = task


In [3]:
def expand_tasks(old_tasks, op):
    return sum(map(lambda tk: PacmanTaskShifter.neighbors(tk, [op]), old_tasks), [])

In [4]:
task_no_ghost = PacmanTaskShifter.neighbors(task, ['del_ghost'])

In [5]:
task21 = expand_tasks(task_no_ghost, 'del_food')
task21 = expand_tasks(task21, 'move_pacman')
print task21

[%%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%v    %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
%v%%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
%v%.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
%v%%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%v    %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
% v   %
%%%%%%%
Score: 0.00
, %%%%%%%
% v   %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%  v  %
%%%%%%%
Score: 0.00
, %%%%%%%
%  v  %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%   v %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.v %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%   v %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%    v%
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%%v%
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %

In [6]:
task2 = expand_tasks(task_no_ghost, 'move_pacman')
task3 = expand_tasks([task], 'move_pacman')
tasks = task21 + task2 + task3
print tasks

[%%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%v    %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
%v%%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
%v%.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
%v%%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%v    %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
% v   %
%%%%%%%
Score: 0.00
, %%%%%%%
% v   %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%  v  %
%%%%%%%
Score: 0.00
, %%%%%%%
%  v  %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%   v %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.v %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%   v %
% %%% %
% %.  %
% %%% %
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%% %
%    v%
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %%% %
% %.  %
% %%%v%
%     %
%%%%%%%
Score: 0.00
, %%%%%%%
%     %
% %

In [7]:
print 'number of tasks', len(tasks)

number of tasks 65


In [8]:
tasks[1]

%%%%%%%
%     %
% %%% %
% %.  %
%v%%% %
%     %
%%%%%%%
Score: 0.00

In [9]:
tasks[30]

%%%%%%%
%     %
% %%% %
% %   %
% %%%v%
%.    %
%%%%%%%
Score: 0.00

### Test Distance / Metric

In [10]:
PacmanTaskDistance.dist_food(tasks[1], tasks[30])

2.0

## Feature Space

In [11]:
def feat_dist(task_a, task_b):
    weights = {
        'dist_food': 1,
        'dist_pacman': 1,
        'dist_num_ghost': 1
    }
    dist = 0.
    for key, val in weights.items():
        dist += val * getattr(PacmanTaskDistance, key)(task_a, task_b)
    return dist

In [12]:
feat_dist(tasks[1], tasks[30])

6.0

## Meta-Feature Extractors / Experts

In [13]:
def mf_nb(im_mem, curr_task):
    avg_im = 0.
    for (task, im) in im_mem[::-1]:
        if task == curr_task:
            break
        dist = feat_dist(task, curr_task)
        avg_im += np.exp(1.-dist) * im
    return avg_im

mf_exs = [mf_nb]

## Learning

In [14]:
state_shape = task.state_shape

def two_layer(states):
    '''
    two layer neural network with same number of hidden units.
    '''
    state_vecs = states.flatten(2)
    return arch.two_layer(state_vecs, np.prod(task.state_shape), 128, task.num_actions)

def conv_net(states):
    '''
    conv+max-pool layer first followed by a fully-connected layer.
    '''
    conv1 = Conv(state_shape[0], output_dim=32, filter_size=(2,2), pool_size=(2,2), activation='relu')
    h1 = conv1(states)
    h1_dim = int((state_shape[1]-2+1)/2) * int((state_shape[2]-2+1)/2) * 32
    h1 = h1.flatten(2)
    fc1 = FullyConnected(input_dim=h1_dim, output_dim=128, activation='relu')
    h2 = fc1(h1)
    linear_layer = FullyConnected(input_dim=128, output_dim=task.num_actions, activation=None)
    output = linear_layer(h2)
    return (output, {
        'conv1': conv1,
        'fc1': fc1
    })



def conv_net2(states):
    '''
    2 conv + max-pool layers followed by a fully-connected layer.
    '''
    conv1 = Conv(state_shape[0], output_dim=16, filter_size=(2,2), pool_size=(2,2), activation='relu')
    h1 = conv1(states)
    h1_shape = (16, int((state_shape[1]-2+1)/2), int((state_shape[2]-2+1)/2))
    print 'h1_shape', h1_shape
    conv2 = Conv(h1_shape[0], output_dim=32, filter_size=(2,2), pool_size=(2,2), activation='relu')
    h2_shape = (int((h1_shape[1]-2+1)/2), int((h1_shape[2]-2+1)/2), 32)
    print 'h2_shape', h2_shape
    h1 = conv1(states)
    h2 = conv2(h1)
    h2 = h2.flatten(2)
    fc1 = FullyConnected(input_dim=np.prod(h2_shape), output_dim=32, activation='relu')
    h2 = fc1(h2)
    linear_layer = FullyConnected(input_dim=32, output_dim=task.num_actions, activation=None)
    output = linear_layer(h2)
    return (output, {
        'conv1': conv1,
        'conv2': conv2,
        'fc1': fc1
    })

In [98]:
dqn = DQN(task, arch_func=conv_net, state_type=T.tensor4)

In [113]:
gamma = 0.9
budget = 20
learner = DeepQlearn(dqn, gamma=gamma, lr=1e-2, memory_size = 500, epsilon=0.3)
def run_task(task, num_iter):
    for it in range(num_iter):
        learner.run(task, num_episodes = 10, tol=1e-4)
        score = reward_stochastic(dqn, task, gamma=gamma, num_trials=10, tol=1e-4, method='eps-greedy', epsilon=0.0)
        scores.append(score)
        total_exps.append(learner.total_exp)
        print 'it', it, 'total exp', learner.total_exp, 'score', scores[-1]
        
train_func = lambda task:  learner.run(task, num_episodes = 10, tol=1e-4, budget=budget)
# eval_func = lambda policy, task: qval_stochastic(policy, task, gamma=gamma, num_trials=10, budget=budget, method='eps-greedy', epsilon=0.)
eval_func = lambda policy, task: reward_stochastic(policy, task, gamma=gamma, num_trials=10, budget=budget, method='eps-greedy', epsilon=0.)
score_func = lambda policy, task: reward_stochastic(policy, task, gamma=gamma, num_trials=10, tol=1e-4, method='eps-greedy', epsilon=0.)

In [114]:
qval_stochastic(dqn, task0, gamma=gamma, num_trials=10, budget=budget, method='eps-greedy', epsilon=0.)

-209.80508623732007

In [115]:
reward_stochastic(dqn, task0, gamma=gamma, num_trials=10, budget=budget, method='eps-greedy', epsilon=0.)

-216.16935261841007

In [102]:
train_func(tasks[0])

In [103]:
eval_func(dqn, tasks[0])

-4.1163537674500068

In [104]:
meta = DQCL.DriftExpert(dqn, mf_exs, train_func, eval_func, eta=1/20.)

In [105]:
def topk(dic, k=5):
    candidates = sorted(dic.keys(), key=lambda key: dic[key], reverse=True)
    return {key: dic[key] for key in candidates[:k]}
    

In [112]:
scores = []
exs = []
for it in range(1000):
    meta.run(tasks, num_epochs=1)
    print 'iteration', it
#     print 'count'
#     pprint(topk(meta.task_count))
#     print 'pred'
#     pprint(topk(meta.im_pred))
#     print 'ucb'
#     pprint(topk(meta.im_ucb))
#     print 'meta features'
#     pprint(topk(meta.im_feat))
#     print 'im_mem'
#     pprint(meta.im_mem)
#     pprint(meta.diagnostics)
    for key in ['score after', 'score before', 'task chosen']:
        print key, meta.diagnostics[key]
    task_chosen = meta.diagnostics['task chosen']
    print 'pred', meta.im_feat[task_chosen][str(mf_nb)]
    if (it + 1) % 10 == 0:
        exs.append(learner.total_exp)
        score = score_func(dqn, task0)
        scores.append(score)
        print '=== game play score ===', score
    print '---------------------------------------'
#     clear_output()

iteration 0
score after 25.7663080986
score before 39.8293455057
task chosen %%%%%%%
%     %
% %%% %
% %.v %
% %%% %
%.    %
%%%%%%%
Score: 0.00

pred 290.74222112
---------------------------------------
iteration 1
score after 46.3818137006
score before 16.3567954866
task chosen %%%%%%%
%     %
% %%% %
% %. v%
% %%% %
%.    %
%%%%%%%
Score: 0.00

pred 228.430580216
---------------------------------------
iteration 2
score after 509.0
score before 509.0
task chosen %%%%%%%
%     %
% %%% %
% %.v %
% %%% %
%     %
%%%%%%%
Score: 0.00

pred 213.346469793
---------------------------------------
iteration 3
score after 55.6740087306
score before 46.1335521821
task chosen %%%%%%%
%     %
% %%% %
% % v %
% %%% %
%.    %
%%%%%%%
Score: 0.00

pred 201.209334725
---------------------------------------
iteration 4
score after 83.180569362
score before 37.3373932445
task chosen %%%%%%%
%     %
% %%% %
% %v  %
% %%% %
%.    %
%%%%%%%
Score: 0.00

pred 160.592181795
---------------------------------

KeyboardInterrupt: 

In [24]:
it

58

In [111]:
task = tasks[50]
# task = task0
task.reset()
while not task.is_end():
    action = dqn.get_action(task.curr_state, method='eps-greedy', epsilon=0., valid_actions=task.valid_actions)
#     action = dqn.get_action(task.curr_state, method='softmax', temperature=1, valid_actions=task.valid_actions)
    task.step(action)
    print task
    print task.valid_actions
    sys.stdout.flush()
    time.sleep(.5)
    clear_output()